In [17]:
import torchvision,torch,time
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from vit_pytorch import ViT

# 设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))
import os
current_directory = os.getcwd()
print("Current working directory:", current_directory)

using cuda device.
Current working directory: d:\CODE\Pycharm\Codes\AI\scripts


In [18]:
dataset_transform = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.FashionMNIST(root="../dataset", train=True, transform=dataset_transform, download=False)
test_data = torchvision.datasets.FashionMNIST(root="../dataset", train=False, transform=dataset_transform, download=False)
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

训练数据集的长度为：60000
测试数据集的长度为：10000


In [20]:
v = ViT(
    image_size=28,
    patch_size=4,
    num_classes=10,
    dim=1024,
    depth=6,
    heads=16,
    mlp_dim=2048,
    dropout=0.1,
    emb_dropout=0.1,
    channels=1
).to(device)
check_input = torch.ones((64, 1, 28, 28)).to(device)
output = v(check_input)
print(output.shape)

loss_fn = nn.CrossEntropyLoss() # 交叉熵损失函数
loss_fn = loss_fn.to(device)
learning_rate = 1e-2
optimizer = torch.optim.SGD(v.parameters(), lr=learning_rate)

torch.Size([64, 10])


In [19]:
total_train_step = 0    # 训练次数
total_test_step = 0     # 测试次数
epoch = 20              # 训练轮数
writer = SummaryWriter("../logs/vit_sample")
last_time = time.time()
for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i+1))
    # 训练
    v.train() # 对某些特定的层需要此句
    for data in train_dataloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = v(inputs)
        loss = loss_fn(outputs, labels)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试
    v.eval() # 对某些特定的层需要此句
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = v(inputs)
            loss = loss_fn(outputs, labels)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == labels).sum() # argmax(0)竖着看，argmax(1)横着看
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(v.state_dict(), "../models/vit_sample/v_{}.pth".format(i))
    delta_time = time.time() - last_time
    last_time = time.time()
    print("模型已保存，耗时：{}".format(delta_time))

writer.close()

训练次数：1500, Loss: 0.8571808338165283


KeyboardInterrupt: 